In [14]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
import numpy as np
import h5py
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
from shapely.geometry import Point

import rasterio
import tables as tab
import tqdm.notebook as tqdm
from skimage.transform import  AffineTransform
from rasterio.transform import Affine
from skimage.measure import ransac
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, SIFT
from PIL import Image
import re
import math, requests
import io
import tqdm
import sys



from PRISMA_georeference import GoogleMapDownloader, GoogleMapsLayers, pixel_to_latlon, latlon_to_pix, prisma_2_tiff, match_subblocks

In [15]:
MATCHING_CUTOFF = 0.7
SEP_CUTOFF = 0.01

def adjust_lat_lon(path_to_PRISMA_file, outpath="", band_for_rgb = 20):
    '''
    path_to_PRISMA_file - should be the full path so that re works
    '''
    zoom_level = 11
    tile_width = 7
    tile_height = 7
    
    
    metrics = {}
    print("# 1 create .tif file")
    tiff_output = outpath + '.tif'
    mean_lat, mean_lon = prisma_2_tiff(path_to_PRISMA_file, tiff_output, band_for_rgb)
    
    print("# 2 Download gmaps for the same location")
    gmd = GoogleMapDownloader(mean_lat, mean_lon, zoom_level,
                              GoogleMapsLayers.SATELLITE)
    img, tile_coords_corner = gmd.generateImage(tile_width=tile_width,
                                                tile_height=tile_height)
    
    print(" 3 Determine matched features in PRISMA image to gMaps image with SIFT")
    tif_coords, gmaps_coords, nmatches = \
        match_subblocks(tiff_output, tile_coords_corner, img, cutoff=MATCHING_CUTOFF)
    metrics['n_matches'] = nmatches
    del img
    
    orig_errs = np.sqrt(np.sum((tif_coords-gmaps_coords)**2, axis=1))
    metrics['orig_mean_error'] = orig_errs.mean()
    
    keep = orig_errs < SEP_CUTOFF
    src = np.array(tif_coords[keep])
    dst = np.array(gmaps_coords[keep])
    
    
    print(" 4 Define mapping between PRISMA LatLon and gMaps latlon")
    mapping, inliers = ransac((src, dst), AffineTransform, min_samples=4,
                               residual_threshold=0.002, max_trials=100)
    new_errs = np.sqrt(np.sum((mapping(src)-dst)**2, axis=1))
    metrics['new_mean_error'] = new_errs.mean()
    
    n_outliers = np.sum(~inliers)
    metrics['n_outliers'] = n_outliers
    
    print(" 5 Map PRISMA Latlons to gMaps latlons (ransac affine) and save for reference")
    orig = rasterio.open(tiff_output)
    adjusted_latlon = mapping(np.vstack([orig.read(2).flatten(),
                                        orig.read(3).flatten()]).transpose())
    orig.close()
    adjusted_latlon = adjusted_latlon.reshape((1000,1000,2))
    #np.savez(outpath + "_adjll.npz", adjusted_latlon = adjusted_latlon)
    np.save(outpath + ".npy", adjusted_latlon)
    return adjusted_latlon, metrics

In [38]:
dir_path = r"C:\Users\simon\Documents\Skole\EIT\Tare_Naturbase"
prisma_path = "HyperSpectral1.he5"
prisma_path = dir_path + "/" + prisma_path
outpath = "out/band_550"

adjusted = adjust_lat_lon(prisma_path, outpath, band_for_rgb=15)

# 1 create .tif file
# 2 Download gmaps for the same location
Image size (pix):  (1792, 1792)
Tile coordinate top left (north-west) corner: 1064,558
 3 Determine matched features in PRISMA image to gMaps image with SIFT


100%|██████████| 81/81 [01:14<00:00,  1.08it/s]


 4 Define mapping between PRISMA LatLon and gMaps latlon
 5 Map PRISMA Latlons to gMaps latlons (ransac affine) and save for reference


In [39]:
file1 = tab.open_file(prisma_path)
d2 = file1.get_node("/HDFEOS/SWATHS/PRS_L1_HRC/")
latv = np.array(d2['Geolocation Fields']['Latitude_VNIR'])
lonv = np.array(d2['Geolocation Fields']['Longitude_VNIR'])
vnir = np.array(d2['Data Fields']['VNIR_Cube'])
print(vnir.shape)
print(latv.shape)
print(lonv.shape)
file1.close()

(1000, 66, 1000)
(1000, 1000)
(1000, 1000)


In [40]:
adjusted_lats = adjusted[0][:, :, 0]
adjusted_lons = adjusted[0][:, :, 1]

In [41]:
columns = []
for i in range(66):
    columns.append(f"band_{i}")
columns.append("lat")
columns.append("lon")

In [42]:
df = pd.DataFrame()

In [43]:
for i, band in enumerate(columns[:-2]):
    print(i)
    df[band] = vnir[:, i, :].flatten()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


In [44]:
df["lat"] = adjusted_lats.flatten()
df["lon"] = adjusted_lons.flatten()

In [45]:
df.head()

,band_0,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,...,band_58,band_59,band_60,band_61,band_62,band_63,band_64,band_65,lat,lon
0,0,0,0,4328,3454,2358,1475,1508,3103,4234,...,4367,4631,4676,4721,4486,4699,5339,5782,62.898613,7.981551
1,0,0,0,4460,3526,2372,1513,1550,3298,4332,...,4893,5198,5248,5002,4540,5106,5969,6424,62.898358,7.981362
2,0,0,0,4634,3717,2507,1546,1664,3421,4516,...,4806,5010,5309,4944,4672,5280,5670,5803,62.898103,7.981174
3,0,0,0,4717,3744,2545,1559,1641,3786,4502,...,4709,5062,5318,4922,4640,5044,5388,6291,62.897852,7.980986
4,0,0,0,4702,3791,2581,1575,1649,3832,4528,...,4892,5266,5268,4938,4659,5099,5685,5871,62.897597,7.980797


In [46]:
df.to_pickle("df_for_malan3.pickle")